In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sympy import *
import itertools
import math

# Problem:
### $\max_{\vec{w}}{F(\vec{w})}$, s.t. $t < T$

# Where,
- $t = \sum_{i}^{N}{w_i}$
- $T \equiv$ Total Time Allottment for Class
- $N \equiv$ Number of Assignments in Class
- $F(\vec{w}) = \sum_{g \in G}{\frac{g_w}{g_N} \sum_{w \in \vec{w_g}, a \in g_A}{F_a(w)}}$
- $F \equiv$ Grade in the Class
- $\vec{w} \equiv$ work time array
- $G \equiv$ Grade Sections in Class
- $g_w \equiv$ Grade Section's Grading Weight
- $g_N \equiv$ Number of Assignment in Grade Section
- $\vec{w_g} \equiv$ Work Time's Relevant to Grade Section
- $g_A \equiv$ Assignments in Grade Section
- $F_a(w) \equiv$ Expected Assignment Grade given Work Time
- $a_r \equiv$ Assignment runtime

## So far, $F_a(w)$ can be two different functions:
- WorkCurve_tanh: $F_a(w) = \tanh(\frac{w}{a_r})$
- WorkCurve_linear: $F_a(w) = \frac{w}{a_r}$

In [2]:
class Assignment:
    
    def __init__(self, name=None, grade=0, assignment_runtime_averages=None, assignment_runtime_std=0.1, work_curve='quadratic'):
        
        self.name = name
        
        if assignment_runtime_averages is None:
            self.assignment_runtime_averages = {
                'HW': {'hours': 5},
                'Exam': {'hours': 3},
                'Quiz': {'minutes': 15},
                'Attendance': {'hours': 1, 'minutes': 30},
                'Project': {'hours': 12},
            }
        else:
            self.assignment_runtime_averages = assignment_runtime_averages
        self.assignment_runtime_std = assignment_runtime_std
        
        self.assignment_type = None
        self.runtime = None
        self.time_spent = 0
        
        self.work_curve_name = work_curve
        self.work_curve = {
            'tanh': self.WorkCurve_tanh,
            'linear': self.WorkCurve_linear,
            'quadratic': self.WorkCurve_quadratic,
        }[work_curve]
        
        self.grade = grade
    
    def __str__(self):
        
        return '| %s: (Runtime: %s) (Time Spent: %s) (Grade: %1.4f%%) \n' % (self.name, self.runtime, datetime.timedelta(seconds=float(self.time_spent)), self.grade*100)
    
    
    def WorkCurve_quadratic(self, w):
        
        return (w / self.runtime_seconds) ** 2
    
    def WorkCurve_tanh(self, w):
        
        return np.tanh(w/self.runtime_seconds)
    
    def WorkCurve_linear(self, w):
        
         return w / self.runtime_seconds
         
    def dFa_dw(self, w):
        
        runtime = self.runtime_seconds
        x = (w/runtime)
                
        func_dict = {
            'tanh': lambda x: (1/runtime)*(((np.exp(-x) + np.exp(x))/2)**(-2)),
            'linear': lambda x: (1/runtime)*(1),
            'quadratic': lambda x: (1/runtime)*(2*x),
            'logistic': lambda x: (1/runtime)*(2*np.exp(-x) / (1 + np.exp(-x))**3),
            
        }
        
        return func_dict[self.work_curve_name](x)
        
    def work_on(self, seconds_working):
            
        self.time_spent = seconds_working
        self.grade = self.work_curve(seconds_working)
        
        return self.grade
    
    def set_runtime(self):
        
        assert self.assignment_type in self.assignment_runtime_averages.keys()
        
        average_runtime = self.assignment_runtime_averages[self.assignment_type]
        average_runtime_datetime = datetime.timedelta(**average_runtime)
        average_total_seconds = average_runtime_datetime.total_seconds()
        
        runtime = datetime.timedelta(seconds=np.random.normal(average_total_seconds, scale=self.assignment_runtime_std*average_total_seconds))
        self.runtime = runtime
        self.runtime_seconds = runtime.total_seconds()
    
    def set_assignment_type(self, assignment_type):
        
        self.assignment_type = assignment_type
        self.set_runtime()
    
    

In [3]:
class GradeSection:
    
    def __init__(self, assignment_type, grading_weight, assignments):
        
        self.assignment_type = assignment_type
        self.grading_weight = grading_weight
        self.assignments = assignments
        self.num_assignments = len(self.assignments)
        
        for assignment in self.assignments:
            
            assignment.set_assignment_type(self.assignment_type)
    
    def __str__(self):
        
        s = '%s (%1.4f%%): \n' % (self.assignment_type, self.get_grade()*100)
        for assignment in self.assignments:
            s += str(assignment)
        s += ('=' * 30) + '\n'
        
        return s
    
    def get_grade(self):
        
        return sum([assignment.grade for assignment in self.assignments]) / self.num_assignments
    
    def get_weighted_grade(self):
        
        return self.grading_weight * self.get_grade()

    def get_total_runtime(self):
        
        return datetime.timedelta(seconds=sum([assignment.runtime.total_seconds() for assignment in self.assignments]))
    


In [4]:
class SchoolCourse:
    
    def __init__(self, grade_sections):
        
        self.grade_sections = grade_sections        
        self.num_grade_sections = len(grade_sections)
        assert sum([grade_section.grading_weight for grade_section in self.grade_sections]) == 1
        
        self.assignments = [assignment for grade_section in self.grade_sections for assignment in grade_section.assignments ]
        self.num_assignments = len(self.assignments)
        
        
    def __str__(self):
        
        s = ''
        for grade_section in self.grade_sections:
            s += str(grade_section)
        
        s += "..." * 10
        s += "\nGRADE: %1.3f%%" % (self.get_grade() * 100)
        s += "\nTOTAL RUNTIME NEEDED FOR an A: %s" % self.get_total_runtime()
        
        return s
    
    def copy(self):
        
        return SchoolCourse(self.grade_sections)
    
    def get_grade(self):
        
        return sum([grade_section.get_weighted_grade() for grade_section in self.grade_sections])
    
    def get_grades(self):
        
        return [assignment.grade for assignment in self.assignments]
    
    def work(self, work_time):
        
        if len(self.assignments) != len(work_time):
            raise RuntimeError('Work time array does not align with assignment array. Work time shape: (%s), Assignment shape: (%s)' % (len(work_time), len(self.assignments)))
        
        for assignment, work_time in zip(self.assignments, work_time):
            assignment.work_on(work_time)
        
        return self.get_grade()
        
    def get_total_runtime(self):
        
        return datetime.timedelta(seconds=sum([grade_section.get_total_runtime().total_seconds() for grade_section in self.grade_sections]))
    

In [5]:
HW_section = GradeSection(
    assignment_type='HW', 
    grading_weight=0.15, 
    assignments=[
        Assignment('HW #1'),
        Assignment('HW #2'),
        Assignment('HW #3'),
        Assignment('HW #4'),
        Assignment('HW #5'),
        Assignment('HW #6'),
    ]
)

exam_section = GradeSection(
    assignment_type='Exam', 
    grading_weight=0.4, 
    assignments=[
        Assignment('Midterm Exam 1'),
        Assignment('Midterm Exam 2'),
        Assignment('Final Exam'),
    ]
)

quiz_section = GradeSection(
    assignment_type='Quiz', 
    grading_weight=0.2, 
    assignments=[
        Assignment('Quiz #1'),
        Assignment('Quiz #2'),
        Assignment('Quiz #3'),
        Assignment('Quiz #4'),
        Assignment('Quiz #5'),
        
    ]
)

attendance_section = GradeSection(
    assignment_type='Attendance', 
    grading_weight=0.125, 
    assignments=[
        Assignment('Attendance #1'),
        Assignment('Attendance #2'),
        Assignment('Attendance #3'),
        Assignment('Attendance #4'),
        Assignment('Attendance #5'),
        Assignment('Attendance #6'),
        Assignment('Attendance #7'),
        Assignment('Attendance #8'),
        Assignment('Attendance #9'),
        Assignment('Attendance #10'),
        
    ]
)

project_section = GradeSection(
    assignment_type='Project', 
    grading_weight=0.125, 
    assignments=[
        Assignment('Report #1'),
        Assignment('Report #2'),
        Assignment('Report #3'),
        
    ]
)

In [6]:
ExampleCourse = SchoolCourse(grade_sections=[
    HW_section,
    exam_section,
    quiz_section,
    attendance_section,
    project_section
])

In [7]:
ExampleCourse.get_grade()

0.0

In [8]:
ExampleCourse.work([datetime.timedelta(minutes=1).total_seconds()]*ExampleCourse.num_assignments)

0.001032657439821029

In [9]:
print(ExampleCourse)

HW (0.0012%): 
| HW #1: (Runtime: 5:00:15.582485) (Time Spent: 0:01:00) (Grade: 0.0011%) 
| HW #2: (Runtime: 4:04:53.936877) (Time Spent: 0:01:00) (Grade: 0.0017%) 
| HW #3: (Runtime: 4:58:49.614034) (Time Spent: 0:01:00) (Grade: 0.0011%) 
| HW #4: (Runtime: 4:52:37.303305) (Time Spent: 0:01:00) (Grade: 0.0012%) 
| HW #5: (Runtime: 5:04:00.006015) (Time Spent: 0:01:00) (Grade: 0.0011%) 
| HW #6: (Runtime: 4:38:04.789198) (Time Spent: 0:01:00) (Grade: 0.0013%) 
Exam (0.0038%): 
| Midterm Exam 1: (Runtime: 2:32:43.110239) (Time Spent: 0:01:00) (Grade: 0.0043%) 
| Midterm Exam 2: (Runtime: 2:29:45.070133) (Time Spent: 0:01:00) (Grade: 0.0045%) 
| Final Exam: (Runtime: 3:15:43.833461) (Time Spent: 0:01:00) (Grade: 0.0026%) 
Quiz (0.5000%): 
| Quiz #1: (Runtime: 0:13:25.823434) (Time Spent: 0:01:00) (Grade: 0.5544%) 
| Quiz #2: (Runtime: 0:13:45.075839) (Time Spent: 0:01:00) (Grade: 0.5288%) 
| Quiz #3: (Runtime: 0:13:45.343712) (Time Spent: 0:01:00) (Grade: 0.5285%) 
| Quiz #4: (Runtime: 0

# ======================

In [10]:
# class OptimizeSchoolCourse:

#     def __init__(self, total_time_allotment, CLASS, alpha=1e-6, max_iterations=1e6, stopping_threshold=1e-3, quick=True):
        
#         self.log = pd.DataFrame({
#             'Iteration #': [],
#             'F(w)': [],
#             '|Gradient F(w)|': [],
#             'w': []
            
#         })
        
#         self.quick = quick
        
#         self.CLASS = CLASS
        
#         self.T = datetime.timedelta(**total_time_allotment)
#         self.T_min = 0
#         self.T_max = self.T.total_seconds()
        
#         self.G = self.CLASS.grade_sections
#         self.A = self.CLASS.assignments
#         self.N = len(self.A)
        
#         self.alpha = alpha
#         self.max_iterations = max_iterations
#         self.stopping_threshold = stopping_threshold
        
#         if self.T >= self.CLASS.get_total_runtime():
            
#             raise RuntimeError("YOU HAVE ENOUGH TIME TO GET A 100%! YOU DON'T NEED AN OPTIMIZER!")
        
#         self.w_symbols = [symbols('w%d' % i) for i in range(self.N)]
#         self.Lambda_symbol = symbols("lambda")
#         self.t_symbols = [symbols('t%d' % i) for i in range(self.N)]
#         self.Theta_symbols = [symbols('theta%d' % i) for i in range(self.N)]
        
#         self.unknowns = self.w_symbols + [self.Lambda_symbol] + self.t_symbols + self.Theta_symbols
    
    
#     def evaluate(self, w_vec):
                
#         return self.CLASS.work(w_vec)
    
#     def get_case(self, m):
        
#         theta_modifiers = format(m, "0%ib" % (self.N + 1))
#         t_modifiers = theta_modifiers.replace('0', 'temp').replace('1', '0').replace('temp', '1')
        
#         theta_modifiers = [int(b) for b in theta_modifiers]
#         t_modifiers = [int(b) for b in t_modifiers]
        
#         return theta_modifiers, t_modifiers
        
#     def gradient_equations(self, sol_case):
        
#         theta_modifiers, t_modifiers = sol_case

#         equations = []
        
#         assignment_num = 0
#         for g in self.G:
            
#             const = g.grading_weight / g.num_assignments
            
#             for i, assignment in enumerate(g.assignments):
                
#                 theta = self.Theta_symbols[assignment_num] * theta_modifiers[assignment_num]
#                 dFai_dwi = assignment.dFa_dw(self.w_symbols[assignment_num])
                
#                 equation = Eq((const - theta) * dFai_dwi - self.Lambda_symbol, 0)
#                 equations.append(equation)
        
#                 assignment_num += 1
        
#         return equations
    
#     def cond1_equations(self, sol_case):
        
#         theta_modifiers, t_modifiers = sol_case

#         equations = [Eq(self.g1(), 0)]
        
#         return equations
    
#     def cond2_equations(self, sol_case):
        
#         theta_modifiers, t_modifiers = sol_case

#         equations = [Eq(self.g2(t_modifiers[i], i), 0) for i in range(self.N)]
        
#         return equations
    
    
#     def equations(self, solution_case):
        
#         gradient_equations = self.gradient_equations(solution_case)
#         condition_1_equations = self.cond1_equations(solution_case)
#         condition_2_equations = self.cond2_equations(solution_case)

#         equations = gradient_equations + condition_1_equations + condition_2_equations
        
#         return equations
    
#     def get_solution(self):
        
#         self.solution = self.solve()
                
#         self.w_vec = np.array([self.solution['w%d' % i] for i in range(self.N)])
        
#         self.Theta_vec = np.array([self.solution['theta%d' % i] for i in range(self.N)])
#         self.Lambda = self.solution['lambda']
#         self.t = self.solution['t']
        
#         return {
#             'w': self.w_vec,
#             'lambda': self.Lambda,
#             't': self.t,
#             'theta': self.Theta_vec,
#         }
        
#     def solve(self):
        
#         solutions = []
#         num_cases = 2**self.N
#         error = ''
        
#         for case_num in range(num_cases):
            
#             print(
#                 'Solution case #%i/%i (%1.2f %%) | # of Solutions: %i | LOG: %s' % (
#                     case_num, num_cases, (case_num/num_cases)*100, len(solutions), error
#                 ), end='\n')
            
#             print('==> Generating equations...', end=' ')
#             solution_case = self.get_case(case_num)
#             equations = self.equations(solution_case)
#             print('DONE.')
            
#             print('==> Solving...', end=' ')
    
#             try:
#                 solutions.append(solve(equations, self.unknowns, simplify=False, dict=True))
#             except Exception as e:
#                 error = e.__str__()
            
#             print('DONE.\n%s' % ('='*30))
            
#         return max(solutions, key=lambda x: self.evaluate(np.array(x[0][:self.N])))
    
#     def u(self, i):
        
#         return self.A[i].work_on(self.w_symbols[i]) - 1
    
#     def g2(self, t_modifier, i):
        
#         return self.u(i) + t_modifier * (self.t_symbols[i] ** 2)
    
#     def g1(self):
        
#         return self.T_max - sum(self.w_symbols)
    
#     def grad_F(self):
        
#         w_vec = self.w_symbols
        
#         gw = np.array([
#             grade_section.grading_weight 
#             for grade_section in self.G
#             for i in range(grade_section.num_assignments) 

#         ])
#         gn = np.array([
#             grade_section.num_assignments 
#             for grade_section in self.G
#             for i in range(grade_section.num_assignments) 
#         ])
        
#         const = gw/gn
#         dFa_dw = np.array([assignment.dFa_dw(wi) for assignment, wi in zip(self.A, w_vec)])
        
#         return const * dFa_dw
    
#     def __str__(self):
        
#         s = 'WORK ARRAY:\n'
        
#         for work_time, assignment in zip(self.w_vec, self.A):
#             s += '| %s: %s\n' % (assignment.name, str(work_time))
        
#         s += 'TOTAL: %s' % str(datetime.timedelta(seconds=sum([float(w) for w in self.w_vec])))
        
#         return s


In [11]:
class OptimizeSchoolCourse:

    def __init__(self, total_time_allotment, CLASS, alpha=1e-6, max_iterations=1e6, stopping_threshold=1e-3, quick=True):
        
        self.log = pd.DataFrame({
            'Iteration #': [],
            'F(w)': [],
            '|Gradient F(w)|': [],
            'w': []
            
        })
        
        self.quick = quick
        
        self.CLASS = CLASS
        
        self.T = datetime.timedelta(**total_time_allotment)
        self.T_min = 0
        self.T_max = self.T.total_seconds()
        
        self.G = self.CLASS.grade_sections
        self.A = self.CLASS.assignments
        self.N = len(self.A)
        
        self.alpha = alpha
        self.max_iterations = max_iterations
        self.stopping_threshold = stopping_threshold
        
        if self.T >= self.CLASS.get_total_runtime():
            
            raise RuntimeError("YOU HAVE ENOUGH TIME TO GET A 100%! YOU DON'T NEED AN OPTIMIZER!")
        
        self.w_vec = self.solve()
        
    def evaluate(self, w_vec):
                
        return self.CLASS.work(w_vec)
    
    def solve(self):
        
        nu = 0
        for g in self.G:
            for a in g.assignments:
                
                nu += (g.num_assignments * a.runtime_seconds**2) / g.grading_weight
        
        L = self.T_max / nu
    
        w_vec = []
        for g in self.G:
            for a in g.assignments:
                
                w_vec.append(L * a.runtime_seconds**2 * g.num_assignments / g.grading_weight)
        
        return np.array(w_vec)
    
    def get_case(self, m):
        
        theta_modifiers = format(m, "0%ib" % (self.N + 1))
        t_modifiers = theta_modifiers.replace('0', 'temp').replace('1', '0').replace('temp', '1')
        
        theta_modifiers = [bool(int(b)) for b in theta_modifiers]
        t_modifiers = [bool(int(b)) for b in t_modifiers]
        
        return theta_modifiers, t_modifiers
    
    def __str__(self):
        
        s = 'WORK ARRAY:\n'
        
        for work_time, assignment in zip(self.w_vec, self.A):
            s += '| %s: %s\n' % (assignment.name, str(work_time))
        
        s += 'TOTAL: %s' % str(datetime.timedelta(seconds=sum([float(w) for w in self.w_vec])))
        
        return s


In [49]:
total_time_allotment = {'days': 2}
OSC = OptimizeSchoolCourse(total_time_allotment, ExampleCourse)

In [50]:
print(OSC)

WORK ARRAY:
| HW #1: 8448.163910727371
| HW #2: 5620.074262427975
| HW #3: 8367.728797316433
| HW #4: 8023.822965016825
| HW #5: 8659.955672372325
| HW #6: 7246.1473735107
| Midterm Exam 1: 409.7819357023512
| Midterm Exam 2: 394.0124359071915
| Final Exam: 673.1111031833486
| Quiz #1: 10.56393007064788
| Quiz #2: 11.074738270357921
| Quiz #3: 11.081930590877777
| Quiz #4: 13.149306408716317
| Quiz #5: 13.217329120313103
| Attendance #1: 1387.3431250101191
| Attendance #2: 1463.6295068094387
| Attendance #3: 1645.3890030175148
| Attendance #4: 1404.274569064016
| Attendance #5: 1346.3724196749263
| Attendance #6: 1422.9296617439468
| Attendance #7: 1708.977760028997
| Attendance #8: 1462.1404386773793
| Attendance #9: 1802.8506699256352
| Attendance #10: 1757.9733621050807
| Report #1: 33436.62199883979
| Report #2: 45295.21516287529
| Report #3: 30764.396631602453
TOTAL: 2 days, 0:00:00


In [51]:
ExampleCourse.work(OSC.w_vec)

0.11244741119450155

In [52]:
print(ExampleCourse)

HW (20.1147%): 
| HW #1: (Runtime: 5:00:15.582485) (Time Spent: 2:20:48.163911) (Grade: 21.9901%) 
| HW #2: (Runtime: 4:04:53.936877) (Time Spent: 1:33:40.074262) (Grade: 14.6288%) 
| HW #3: (Runtime: 4:58:49.614034) (Time Spent: 2:19:27.728797) (Grade: 21.7808%) 
| HW #4: (Runtime: 4:52:37.303305) (Time Spent: 2:13:43.822965) (Grade: 20.8856%) 
| HW #5: (Runtime: 5:04:00.006015) (Time Spent: 2:24:19.955672) (Grade: 22.5414%) 
| HW #6: (Runtime: 4:38:04.789198) (Time Spent: 2:00:46.147374) (Grade: 18.8614%) 
Exam (0.2403%): 
| Midterm Exam 1: (Runtime: 2:32:43.110239) (Time Spent: 0:06:49.781936) (Grade: 0.2000%) 
| Midterm Exam 2: (Runtime: 2:29:45.070133) (Time Spent: 0:06:34.012436) (Grade: 0.1923%) 
| Final Exam: (Runtime: 3:15:43.833461) (Time Spent: 0:11:13.111103) (Grade: 0.3285%) 
Quiz (0.0192%): 
| Quiz #1: (Runtime: 0:13:25.823434) (Time Spent: 0:00:10.563930) (Grade: 0.0172%) 
| Quiz #2: (Runtime: 0:13:45.075839) (Time Spent: 0:00:11.074738) (Grade: 0.0180%) 
| Quiz #3: (Run

In [53]:
ExampleCourse.get_grade()

0.11244741119450155